In [1]:
import sys, os, multiprocessing, urllib3, csv
from PIL import Image
from io import BytesIO
from tqdm  import tqdm
import json

In [2]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [3]:
client = urllib3.PoolManager(500)

In [4]:
def download_image(item):

    image_id, url = item['imageId'], item['url']
    filename = os.path.join(f"{out_dir}/{image_id}.jpg")

    if os.path.exists(filename): return

    try:
        global client
        response = client.request('GET', url)#, timeout=30)
        image_data = response.data
    except:
        print('Warning: Could not download image %s from %s' % (key, url))
        return

    try:
        pil_image = Image.open(BytesIO(image_data))
    except:
        print('Warning: Failed to parse image %s %s' % (key,url))
        return

    try:
        pil_image_rgb = pil_image.convert('RGB')
    except:
        print('Warning: Failed to convert image %s to RGB' % key)
        return

    try:
        pil_image_rgb.save(filename, format='JPEG', quality=90)
    except:
        print('Warning: Failed to save image %s' % filename)

        return

In [5]:
data_file = '../data/train.json'
out_dir = '../data/kaggle_train'

In [6]:
j = json.load(open(data_file))

In [7]:
j['annotations'][0]

{'labelId': ['95', '66', '137', '70', '20'], 'imageId': '1'}

In [8]:
j['images'][0]

{'url': 'https://contestimg.wish.com/api/webimage/570f35feb2f4b95d223aa9b1-large',
 'imageId': '1'}

In [9]:
pool = multiprocessing.Pool(processes=12)

In [10]:
with tqdm(total=len(j['images'])) as bar:
    for _ in pool.imap_unordered(download_image, j['images']):
        bar.update(1)

 31%|███       | 312613/1014544 [49:19<1:50:44, 105.65it/s]


KeyboardInterrupt: 